# Importação das bibliotecas

Sendo executado no ambiente virtual: teste.

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

# Abertura do arquivo

O arquivo aberto é diário referente ao ano de 2021.

In [ ]:
ds = xr.open_dataset('./dados/dados_ambientais.nc')

In [ ]:
ds

In [ ]:
type(ds)

# Declarando variáveis de interesse

In [ ]:
prec = ds.prec[:,0,0]
nf = ds.nf[:,0,0]
rh = ds.rh2m[:,0,0]
tmin = ds.tmin[:,0,0]
tmed = ds.tmed[:,0,0]
tmax = ds.tmax[:,0,0]

# Criando um array

In [ ]:
array = np.column_stack((prec, nf, rh, tmin, tmed, tmax))

# Criando um DataFrame para dados diários

In [ ]:
ds_diario = pd.DataFrame(array, index=ds.prec.time, columns=['Chuva', 'Focos', 'RH', 'Tmin', 'Tmed', 'Tmax'])
ds_diario.index.name = 'Dia' # Define o nome do index.

In [ ]:
ds_diario

# Converte dados diários para mensal

In [ ]:
# Converte o dados diário para mensal.
ds_mensal = ds_diario.resample('MS').agg({'Chuva':np.sum, 'Focos':np.sum, 'RH':np.mean, 'Tmin':np.mean, 'Tmed':np.mean, 'Tmax':np.mean})
ds_mensal.index.name = 'Mes' # Define o nome do index.

ds_mensal

# Geração dos gráficos

In [ ]:
import cufflinks as cf
from IPython.display import display, HTML

#making all charts public and setting a global theme
cf.set_config_file(sharing='public', theme='white', offline=True)

## Gráfico de linha

In [ ]:
ds_diario[['Tmin', 'Tmed', 'Tmax']].iplot(kind='line', width=1)

In [ ]:
ds_mensal[['Tmin', 'Tmed', 'Tmax']].iplot(kind='line', width=3)

In [ ]:
ds_mensal['Tmed'].iplot(mode='lines+markers', size=4, colors="red", gridcolor="black", 
                        xTitle="Mês", yTitle="(°C)", title="Temperatura")

### Painel de figuras

In [ ]:
ds_mensal[['Tmin', 'Tmed', 'Tmax']].iplot(y=["Tmin", "Tmed", "Tmax"],
                  width=2.0,
                  subplots=True,
                  colors=['blue','black','red'],
                  xTitle="Mês", yTitle="(°C)", title="Temperatura")

## Gráfico de barras vertical

In [ ]:
ds_mensal['Chuva'].iplot(kind='bar')

In [ ]:
ds_mensal[['Chuva', 'RH']].iplot(kind='bar',
                                 bargap=0.5,
                                 dimensions=(1200,600),
                                 xTitle="Mês", yTitle="Prec, RH", title="RH e Prec")

## Gráfico de barras horizontal

In [ ]:
ds_mensal.iplot(kind="bar", y="Chuva",
                yTitle="Mês", xTitle="mm/mês", title="Titulo da figura",
                colors=["tomato"], bargap=0.5,
                sortbars=True,
                dimensions=(1000, 500),
                orientation="h")

## Gráfico de espalhamento

In [ ]:
ds_diario.iplot(kind='scatter', x='RH', y='Tmed', mode='markers')

In [ ]:
ds_diario.iplot(kind="scatter",
              x="RH", y='Tmed',
              mode='markers',
              colors="tomato", size=8, symbol="circle-open-dot",
              bestfit=True, bestfit_colors=["dodgerblue"],
              xTitle="(°C)", yTitle="(%)",
              title="RH x Tmed")

## Gráfico de boxplot

In [ ]:
ds_diario.iplot(kind='box', boxpoints="outliers")

## Gráfico de áreas

In [ ]:
ds_diario.Tmed.iplot(y="Tmed",
               fill=True,
               xTitle="Dia", yTitle="(°C)", title="Temperatura média diária - 2021",
               )

In [ ]:
ds_diario[['Tmin', 'Tmed', 'Tmax']].iplot(
               keys=["Tmin", "Tmed", "Tmax",],
               subplots=True,
               colors=['blue', 'black', 'red'],
               fill=True,
               xTitle="Dia", yTitle="(°C)", title="Temperatura diária - 2021")

## Gráfico de histograma

In [ ]:
ds_diario.Tmed.iplot(kind="hist",
                      bins=50, colors=["red"],
                      keys=["Tmed"],
                      dimensions=(600, 400),
                      title="Histograma de temperatura")

## Gráfico de bolhas

In [ ]:
import plotly.express as px

In [ ]:
ds_diario = ds_diario.reset_index()

In [ ]:
Mes = pd.DatetimeIndex(ds_diario['Dia']).month  # Guarda apenas o mês (1, .., 12).

fig = px.scatter(ds_diario, x="Dia", y="Chuva", size='Chuva', color=Mes, 
                 title = 'Chuva diária')

fig.write_html("file.html")  # Salva no formato HTML.

fig.show()

### Painel de gráfico de bolhas

In [ ]:
# Dado mensal desde 2019 a 2021.
ds = xr.open_dataset('./dados/chuva.2019.to.2021.nc')

In [ ]:
prec = ds.PREC_surface[:,0,0]

In [ ]:
df_prec = pd.DataFrame(prec, index=prec.time, columns=['Chuva'])
df_prec.index.name = 'Mes' # Define o nome do index.

In [ ]:
df_prec = df_prec.reset_index()

In [ ]:
import plotly.express as px

Mes = pd.DatetimeIndex(df_prec['Mes']).month  # Guarda apenas o mês (1, .., 12).
Ano = pd.DatetimeIndex(df_prec['Mes']).year  # Guarda apenas o ano.


In [ ]:
#help(px.scatter)

In [ ]:
fig = px.scatter(df_prec, x='Mes', y='Chuva', size='Chuva', color=Mes, facet_row=Ano)
fig.show()